In [ ]:
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

from mmseg.apis import init_model, inference_model

config_file = "configs/deeplabv3plus/deeplabv3plus_custom.py"
checkpoint_file = "work_dirs/openearth/deeplabv3/apr_16_160k/iter_16000.pth"


# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda')

categories = {
    'unknown': {
        'index': 0, 
        'color': [0,0,0]
    },
    'bareland': {
        'index': 1, 
        'color': [128,0,0]
    },
    'grass': {
        'index': 2, 
        'color': [0,255,36]
    },
    'pavement': {
        'index': 3, 
        'color': [148,148,148]
    },
    'road': {
        'index': 4, 
        'color': [255,255,255]
    },
    'tree': {
        'index': 5, 
        'color': [34,97,38]
    },
    'water': {
        'index': 6, 
        'color': [0,69,255]
    },
    'cropland': {
        'index': 7, 
        'color': [75, 181, 73]
    },
    'buildings': {
        'index': 8, 
        'color': [222, 31, 7]
    }
}

classes = [None] * len(categories)

for key in categories: 
    classes[categories[key]['index']] = key

palette = [categories[key]['color'] for key in classes]

class_labels = {
            "unknown": np.array([0, 0, 0]),
            "Bareland": np.array([128, 0, 0]),
            "Grass": np.array([0, 255, 36]),
            "Pavement": np.array([148, 148, 148]),
            "Road": np.array([255, 255, 255]),
            "Tree": np.array([34, 97, 38]),
            "Water": np.array([0, 69, 255]),
            "Cropland": np.array([75, 181, 73]),
            "buildings": np.array([222, 31, 7]),
        }

key = "hurricane-florence_00000063"

before_key = f"{key}_pre_disaster"
after_key = f"{key}_post_disaster"


before_img_path = f'openearth/xbd/images/{before_key}.png'
after_img_path = f'openearth/xbd/images/{after_key}.png'

before_img_true = Image.open(before_img_path)
after_img_true = Image.open(after_img_path)

def predict(img_path):
    data_sampler = inference_model(model, img_path)
    result = data_sampler.get('pred_sem_seg').data.cpu().numpy()

    result = result.reshape(512, 512, 1).astype(np.uint8)
    result = np.squeeze(result, axis=2)  

    # Load the true mask
    seg_img = Image.fromarray(result.astype(np.uint8)).convert('P')
    seg_img.putpalette(np.array(palette, dtype=np.uint8))
    seg_img = np.array(seg_img.convert('RGB'))
    return seg_img


before_img = predict(before_img_path)
after_img = predict(after_img_path) 

import cv2
import numpy as np

# img = cv2.imread('before.png')

# # Convert the image to the RGB color space if it is not already in that format
# if len(img.shape) > 2 and img.shape[2] == 4:
#     img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
# else:
#     img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

print(np)

def get_unique_colors(img):
    img = np.asarray(img)
    # Calculate the unique colors in the image
    unique_colors = np.unique(img.reshape(-1, img.shape[-1]), axis=0)
    before_colors = {}

    for color in unique_colors:
        for key, value in class_labels.items():
            if (color[0] <= value[0]+40 and color[0] >= value[0]-40 and color[1] <= value[1]+40 and color[1] >= value[1]-40 and color[2] <= value[2]+40 and color[2] >= value[2]-40).all():
                # Create a binary mask for the current color
                mask = np.where(np.all(img == color, axis=-1), 1, 0)

                # Calculate the area of the mask
                area = np.sum(mask)

                # Apply the binary mask to the input image
                masked_img = img * mask[..., np.newaxis]
                before_colors[key]=[masked_img,area]
    return before_colors

before_colors = get_unique_colors(before_img)
after_colors = get_unique_colors(after_img)

#fig, axs = plt.subplots(7, 1, figsize=(24,24))
#current_ax_idx = 0

# Stores the RSCD overlays 
overlays = {}

for k1, v1 in before_colors.items():
    for k2, v2 in after_colors.items():
        if k1 == k2:
            img1 = before_colors[k1][0]
            img2 = after_colors[k1][0]
            img1 = cv2.convertScaleAbs(img1)
            img2 = cv2.convertScaleAbs(img2)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


              # Resize the images to the same size
            img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]))

              # Subtract one array from the other
            diff1 = img1.astype(np.int16) - img2.astype(np.int16)
            diff2 = img2.astype(np.int16) - img1.astype(np.int16)

              # Create binary masks for the lighter pixels
            mask1 = np.zeros_like(diff1)
            mask2 = np.zeros_like(diff2)
            _, mask1_thresh = cv2.threshold(diff1, 20, 255, cv2.THRESH_BINARY)
            mask1[mask1_thresh == 255] = 255
            _, mask2_thresh = cv2.threshold(diff2, 20, 255, cv2.THRESH_BINARY)
            mask2[mask2_thresh == 255] = 255


              # Create a 3-channel RGB image
            result = np.zeros((diff1.shape[0], diff1.shape[1], 3), dtype=np.uint8)
            # Set the red and green channels to the binary masks
            result[mask1 == 255, 0] = 255
            result[mask2 == 255, 1] = 255
            
            overlays[k1] = result
              
            cv2.waitKey(0)
            cv2.destroyAllWindows()

plt.imshow(before_img_true)
plt.imshow(overlays['Cropland'], cmap='jet', alpha=0.4)

1.7.1 True
1.0.0
